# Variational Autoencoder *******
By [Ga Wu](http://wuga214.github.io/) @ University of Toronto

## Introduction
The code is generated to test a hypothesis that we can use End-to-End neural network to do ********. 

The Data in this experiments is generated from RDDL simulator [Github](https://github.com/ssanner/rddlsim), which is written by Prof.Scott Sanner at University of Toronto.

The code is arranged as follow:
1. Write and train Variational Autoencoder transition function, that maps (STATE,ACTION)->(STATE').
2. Write extraction and reloading function for learned weights and bias

### Import Packages
We do note provide pip installation commands, please search this package and install it through pip install. Please upgrade your pip before installing, since old pip would cause errors.

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial

### Load Data
We load data from csv files. The following code shows how to load data through pandas and numpy. Result of this progress can be feed into tensorflow with "feed_dict" argument. 

In [2]:
Datapath="DATA/Navigation/Nav_RDDL_Data.txt"
Labelpath="DATA/Navigation/Nav_RDDL_Label.txt"

In [3]:
#Given local path, find full path
def PathFinder(path):
    #python 2
    #script_dir = os.path.dirname('__file__')
    #fullpath = os.path.join(script_dir,path)
    #python 3
    fullpath=os.path.abspath(path)
    print(fullpath)
    return fullpath

#Read Data for Deep Learning
def ReadData(path):
    fullpath=PathFinder(path)
    return pd.read_csv(fullpath, sep=',', header=0)

#Input Normalization
def Normalize(features, mean = [], std = []):
    if mean == []:
        mean = np.mean(features, axis = 0)
        std = np.std(features, axis = 0)
#     print std
#     print std[:,None]
    new_feature = (features.T - mean[:,None]).T
    new_feature = (new_feature.T / std[:,None]).T
    new_feature[np.isnan(new_feature)]=0
#     print new_feature
    return new_feature, mean, std

In [4]:
x_pd=ReadData(Datapath)
y_pd=ReadData(Labelpath)

/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Nav_RDDL_Data.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Nav_RDDL_Label.txt


In [5]:
x_matrix,_,_=Normalize(x_pd.as_matrix())
x_matrix[:5]
y_matrix=y_pd.as_matrix()
y_matrix[:5]

array([[ 0.11283488,  0.3357079 ],
       [ 0.97125869,  1.23751182],
       [ 0.97125869,  1.99078795],
       [ 0.97125869,  1.99078795],
       [ 1.31838443,  1.03654978]])

In [6]:
data_size,INPUT_SIZE=x_matrix.shape
_,OUTPUT_SIZE = y_matrix.shape

### Support Functions
The following functions allows us passthrough multiple functions without explicitly assign intermediate output variables.

In [9]:
def compose(f,g):
    return lambda x:g(f(x))
    
def composeAll(*args):
    """
    composeAll([f,g,h])(x): f(g(h(x)))
    """
    return partial(functools.reduce, compose)(*args)

## Tensorflow
### Input tensor place holders

In [7]:
# Input features
x = tf.placeholder(tf.float32,[None, INPUT_SIZE],name="Features")

# Input labels
y = tf.placeholder(tf.float32, [None, OUTPUT_SIZE],name="Labels")

# Dropout
dropout = tf.placeholder(tf.float32, name="Dropout")

### Variable Generating functions

In [8]:
#Weight constructing function
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.005)
    return tf.Variable(initial,name="weights")

#Bias constructing function
def bias_variable(shape):
    initial = tf.constant(0.,shape=shape)
    return tf.Variable(initial,name="biases")

### Fully Connected Layer Defination

In [10]:
class Dense():
    """Fully Connected Layer"""
    def __init__(self, scope="fully_connected_layer", output_dim =None, dropout=1.0, activation=tf.identity):
        assert output_dim, "Missing output dimension specification!"
        self.scope = scope
        self.output_dim = output_dim
        self.dropout = dropout
        self.activation = activation
        
    def __call__(self,x):
        with tf.name_scope(self.scope):
            while True:
                try:
                    return self.activation(tf.matmul(x,self.w)+self.b)
                except(AttributeError):
                    self.w = tf.nn.dropout(weight_variable([x.get_shape()[1].value, self.output_dim]),self.dropout)
                    self.b = bias_variable([self.output_dim])
    
    def set_parameters(self, weight, bias):
        self.w.assign(weight)
        self.b.assign(bias)
        
    def get_l2_loss(self):
        return tf.nn.l2_loss(self.w)

### (Conditional) Variational AutoEncoder class
Please notice that the commented code are used to compute traditional VAE, the uncommented one is used to compute conditional VAE.

The following class define a complete VAE network, which include:
1. Network structure specification
2. Loss function specification
3. Prediction specification
4. Optimization method specification
5. Training function
6. Saving function(not tensorflow variable saving, but numpy weight dumping!)
7. Loading function(not tensorflow variable loading, but numpy weight assignment!)
8. Mini-Batch generation function

In [11]:
class ConditionalVAE(object):
    """
    This is a complete implementation of conditional variational autoencoder.
    
    When training, the ConditionalVAE optimize all parameters
    
    """
    
    def __init__(self, 
                 x, #Input Features,I want it can be both variable and placeholder
                 y, #True Label, placeholder
                 dropout, #Drop out
                 num_layers, #number of layers for both encoder and decoder
                 num_hidden_nodes, #number of nodes in each layer
                 activation, #nonlinear activation function
                 learning_rate=0.001, #Learning rate
                 batch_size=100, 
                 l2_lambda = 1E-4): 
        self.x = x
        self.y = y
        self.num_layers = num_layers
        self.num_hidden_nodes = num_hidden_nodes
        self.activation = activation
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.l2_lambda = l2_lambda
        self.dropout = dropout
        self._p_create_vae_graph()
        self._p_create_loss()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        
    
    def _p_create_vae_graph(self):
        layers = []

        #Encode
        encode_layers = self._p_encode()
        std = tf.exp(self.logvar) #This exponential function makes backpropagation through logvar difficult

        #Add encoder denses into layer list
        layers = layers+encode_layers

        #Sample latent value
        eps = tf.random_normal(tf.shape(std),0,1, name='epsilon')
        z = self.mu+tf.mul(tf.mul(std,eps),self.gate) #We multiply a "gate" to help to reduce the variance

        #Decode
        #last_layer_of_x = Dense("decode_x_"+str(self.num_layers),self.x.get_shape()[1].value,self.dropout)
        last_layer_of_y = Dense("decode_y_"+str(self.num_layers),self.y.get_shape()[1].value,self.dropout)
        #decode_x,decode_x_layers = self._p_decode(z,"x")
        #x_pred = last_layer_of_x(decode_x)
        decode_y,decode_y_layers = self._p_decode(z,"y")
        y_pred = last_layer_of_y(decode_y)

        #Add STATE,ACTION decoder denses into layer list
        #layers = layers+decode_x_layers
        #layers.append(last_layer_of_x)

        #Add STATE' decoder dense into into layer list
        layers = layers+decode_y_layers
        layers.append(last_layer_of_y)
        self.layers = layers
        #self.x_pred = x_pred
        self.y_pred = y_pred
        self.std = std #need show the value of std with training process
    
    def _p_encode(self):
        encode_layers = []
        for i in range(self.num_layers):
            encode_layers.append(Dense("encode_"+str(i),self.num_hidden_nodes,self.dropout,self.activation))
        h_encoded = composeAll(encode_layers)(self.x)
        mean_layer = Dense("z_mean",10,self.dropout)
        logvar_layer = Dense("z_log_var",10,self.dropout)
        gate_layer = Dense("z_gate",10,self.dropout,tf.nn.sigmoid)
        mu = mean_layer(h_encoded) #linear activation
        logvar = logvar_layer(h_encoded) #linear activation
        gate = gate_layer(h_encoded)
        encode_layers.append(mean_layer)
        encode_layers.append(logvar_layer)
        encode_layers.append(gate_layer)
        self.mu = mu
        self.logvar = logvar 
        self.gate = gate
        return encode_layers
    
    def _p_decode(self,z,decode_type="x"):
        decode_layers = []
        for i in range(self.num_layers):
            decode_layers.append(Dense("decode_"+decode_type+"_"+str(i),self.num_hidden_nodes,self.dropout,self.activation))
        h_decoded = composeAll(decode_layers)(z)
        return h_decoded,decode_layers
    
    def _p_create_loss(self): #lambda for l2 regularization

        #L2 regularization loss
        l2_loss = tf.constant(0.0)
        for layer in self.layers:
            l2_loss += layer.get_l2_loss()

        #KL distance loss
        kld = -0.5*tf.reduce_sum(1+self.logvar-tf.square(self.mu)-tf.exp(self.logvar),reduction_indices=1)

        #Mean Squared Error
        #mse_x = tf.reduce_mean(tf.square(tf.sub(self.x,self.x_pred)), reduction_indices=1)
        mse_y = tf.reduce_mean(tf.square(tf.sub(self.y,self.y_pred)), reduction_indices=1)

        #loss
        #self.loss = tf.reduce_mean(mse_x+mse_y+kld)+self.l2_lambda*l2_loss
        #self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.reconstructloss = tf.reduce_mean(mse_y)
        #self.loss1 = tf.reduce_mean(mse_x+kld)#+self.l2_lambda*l2_loss
        #self.optimizer1 = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss1)
        self.loss2 = tf.reduce_mean(mse_y+kld)+self.l2_lambda*l2_loss
        self.optimizer2 = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss2)
        
    def train_model(self,data_feature,data_label,dropout_value,epoch=100):
        
        
        summary_writer = tf.train.SummaryWriter('experiment', graph=self.sess.graph)

        #Training
        feed_whole = {self.x: data_feature,self.y: data_label,self.dropout: 1.0}
        new_loss = self.sess.run([self.reconstructloss],feed_dict=feed_whole)
        print('Loss in epoch {0}: {1}'.format("Initial", new_loss)) 
        for epoch in range(epoch):
            batches = self._p_get_batches(data_feature,data_label,self.batch_size)
            for step in range(len(batches)):
                feed_dict = {self.x: batches[step][0],self.y: batches[step][1],self.dropout: dropout_value}
                #training = self.sess.run([self.optimizer1], feed_dict=feed_dict)
                training = self.sess.run([self.optimizer2], feed_dict=feed_dict)
            if epoch%10 == 0:
                new_loss = self.sess.run([self.reconstructloss],feed_dict=feed_whole)
                print('Loss in epoch {0}: {1}'.format(epoch, new_loss))     
        
    def _p_get_batches(self,x_matrix,y_matrix,batch_size):
        remaining_size = len(x_matrix)
        indecs=np.random.permutation(remaining_size)
        x_matrix_perm=x_matrix[indecs]
        y_matrix_perm=y_matrix[indecs]
        batch_index=0
        batches = []
        while(remaining_size>0):
            batch = []
            if remaining_size<batch_size:
                batch.append(x_matrix_perm[batch_index*batch_size:-1])
                batch.append(y_matrix_perm[batch_index*batch_size:-1])
            else:
                batch.append(x_matrix_perm[batch_index*batch_size:(batch_index+1)*batch_size])
                batch.append(y_matrix_perm[batch_index*batch_size:(batch_index+1)*batch_size]) 
            batch_index+=1
            remaining_size-=batch_size
            batches.append(batch)
        return batches
    
    def _p_extract_weights(self):
        # a hashmap maps from layer name to weights and biases
        mp_layer_weights = {}

        #iteratively save values
        for dense in self.layers:
            values = {'weights':dense.w, 'biases':dense.b}
            mp_layer_weights[layer.scope] = values

        return mp_layer_weights
    
    def save_weights(self,path):
        #extract weights from trained model
        layer_weights = self.sess.run(_p_extract_weights())
        print('Whole layer weights: {0}'.format(layer_weights))
        np.save(path,layer_weights)
    
    def load_weights(self,path):
        layer_weights = np.load(path)
        for dense in self.layers:
            print('Scope:{0}'.format(dense.scope))
            values = layer_weights.get(dense.scope)
            weights = values.get('weights')
            biases = values.get('biases')
            dense.set_parameters(weights,biases)
        print('Done!')
    

In [12]:
#Instantiate a network
vae_inst = ConditionalVAE(x,y,dropout,2,20,tf.nn.elu)

### Tensorflow graph visualization function

In [13]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
            batch_index+=1
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:960px;height:600px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [14]:
show_graph(tf.get_default_graph().as_graph_def())

### Training

In [15]:
vae_inst.train_model(x_matrix,y_matrix,1,100)

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Loss in epoch Initial: [6.6099343]
Loss in epoch 0: [0.65296131]
Loss in epoch 10: [0.50756526]
Loss in epoch 20: [0.50848341]
Loss in epoch 30: [0.50753325]
Loss in epoch 40: [0.5091635]
Loss in epoch 50: [0.50731814]
Loss in epoch 60: [0.50595778]
Loss in epoch 70: [0.5050481]
Loss in epoch 80: [0.0080496408]
Loss in epoch 90: [0.0055491212]
